## CH7 앙상블 학습과 랜덤 포레스트

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [53]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

X, y = make_moons (n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## 보팅

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [46]:

log_clf = LogisticRegression ()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True) #소프트 보팅 필수!!!!!

voting_clf = VotingClassifier(
estimators=[ ('lr', log_clf) , ( 'rf' , rnd_clf) , ('svc' , svm_clf ) ], 
    voting= 'hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))])

In [22]:
from sklearn.metrics import accuracy_score
for clf in ( log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit( X_train , y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__ , accuracy_score(y_test, y_pred))

LogisticRegression()

LogisticRegression 0.8666666666666667


RandomForestClassifier()

RandomForestClassifier 0.9333333333333333


SVC()

SVC 0.9666666666666667


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

VotingClassifier 0.9


#### 책에서는 보팅이 가장 높은데 왜.....난....?????

In [47]:
#간접 투표
voting_clf = VotingClassifier(
estimators=[ ('lr', log_clf) , ( 'rf' , rnd_clf) , ('svc' , svm_clf ) ], 
    voting= 'soft')#, probability=True))
voting_clf.fit(X_train, y_train)

for clf in ( log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit( X_train , y_train)
    pred = clf.predict(X_test)
    class_name= clf.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test , pred)))
    #y_pred = clf.predict_proba(X_test) 

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

LogisticRegression()

LogisticRegression 정확도: 0.8000


RandomForestClassifier()

RandomForestClassifier 정확도: 0.9667


SVC(probability=True)

SVC 정확도: 0.9667


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

VotingClassifier 정확도: 0.9667


## 배깅
* 결정트리 분류기 500개의 앙상블을 훈련시킴. 중복 허용 랜덤 70개 샘플 훈련

In [63]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier() , n_estimators=500,
    max_samples=70, bootstrap=True, n_jobs=-1 )
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)



BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=70,
                  n_estimators=500, n_jobs=-1)

In [66]:
#oob 샘플: 평균적으로 선택되지 않은 37%의 훈련 샘플
bag_clf = BaggingClassifier(
DecisionTreeClassifier() , n_estimators=500,
bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_ 

# BaggingClassifier는 태스트 세트에서 약 87.1 %의 정확도를 얻을 것

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=500,
                  n_jobs=-1, oob_score=True)

0.8714285714285714

In [67]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred) 

1.0

In [69]:
# oob_decision_function_: 결정 힘수는 각 훈련 샘플의 클래스 획률을 반환

bag_clf.oob_decision_function_
# 음성 클래스 속할 확률, 양성 클래스 속할 확률 순

array([[1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.95522388, 0.04477612],
       [0.        , 1.        ],
       [0.00584795, 0.99415205],
       [0.86387435, 0.13612565],
       [0.        , 1.        ],
       [0.08791209, 0.91208791],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.95597484, 0.04402516],
       [0.85714286, 0.14285714],
       [1.        , 0.        ],
       [0.95811518, 0.04188482],
       [0.015625  , 0.984375  ],
       [0.04787234, 0.95212766],
       [0.45128205, 0.54871795],
       [0.        , 1.        ],
       [0.26470588, 0.73529412],
       [0.84567901, 0.15432099],
       [0.14594595, 0.85405405],
       [0.8700565 , 0.1299435 ],
       [0.40641711, 0.59358289],
       [0.79365079, 0.20634921],
       [0.        , 1.        ],
       [0.00543478, 0.99456522],
       [0.9       , 0.1       ],
       [0.

* 랜덤 포레스트

In [72]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1 )
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf . predict (X_test)

#BaggingClassifier로 비슷하게 만든 코드
'''
bag_clf = BaggingClassifier(
DecisionTreeClassifier(max_features="auto", max_ leaf_nodes=16),
n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)
'''

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1)

In [74]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data" ], iris["target"])
for name, score in zip( iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)
    
    

RandomForestClassifier(n_estimators=500, n_jobs=-1)

sepal length (cm) 0.09720827232051045
sepal width (cm) 0.02508875167587757
petal length (cm) 0.430874845250884
petal width (cm) 0.4468281307527281


* 이를 통해 petal width와 petal length가 중요한 특성임을 알 수 있다.

## 부스팅
* 에이다부스트
* 그레이디언트 부스팅

In [75]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(
DecisionTreeClassifier(max_depth=1), n_estimators=200,
algorithm= "SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [82]:
from sklearn.tree import DecisionTreeRegressor
tree_reg1= DecisionTreeRegressor(max_depth=2)
tree_reg1 . fit (X, y)

DecisionTreeRegressor(max_depth=2)

In [83]:
y2 = y- tree_reg1 . predict (X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2 . fit (X, y2)

DecisionTreeRegressor(max_depth=2)

In [84]:
y3 = y2 - tree_reg2 . predict (X)
tree_reg3 = DecisionTreeRegressor (max_depth=2)
tree_reg3.fit (X, y3)

DecisionTreeRegressor(max_depth=2)

In [89]:
#y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [88]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor ( max_depth =2 ,n_estimators=3, learning_rate=1.0)
gbrt . fit (X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

* GBRT 앙상블 훈련

In [91]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val =train_test_split (X, y)
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt . fit ( X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt . staged_predict (X_val ) ]

bst_n_estimators = np.argmin(errors) + 1
gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best . fit (X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=120)

GradientBoostingRegressor(max_depth=2, n_estimators=29)

* 연속해서 다섯 번의 반복 동안 검증 오차가 향상되지 않으면 훈련을 멈춤니다.

In [94]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)
#warm_ start =True로 설정하면 사이킷런이 fit () 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 해줍니다.
min_val_error = float ("inf")
error_going_up = 0
for n_estimators in range (1 , 120):
    gbrt .n_estimators =n_estimators
    gbrt .fit (X_train, y_train)
    y_pred = gbrt .predict (X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error :
        min_val_error= val_error
        error_going_up = 0
    else :
        error_going_up += 1
        if error_going_up == 5:
            break # 조기 종료

GradientBoostingRegressor(max_depth=2, n_estimators=1, warm_start=True)

GradientBoostingRegressor(max_depth=2, n_estimators=2, warm_start=True)

GradientBoostingRegressor(max_depth=2, n_estimators=3, warm_start=True)

GradientBoostingRegressor(max_depth=2, n_estimators=4, warm_start=True)

GradientBoostingRegressor(max_depth=2, n_estimators=5, warm_start=True)

...

GradientBoostingRegressor(max_depth=2, n_estimators=23, warm_start=True)

GradientBoostingRegressor(max_depth=2, n_estimators=24, warm_start=True)

GradientBoostingRegressor(max_depth=2, n_estimators=25, warm_start=True)

GradientBoostingRegressor(max_depth=2, n_estimators=26, warm_start=True)

GradientBoostingRegressor(max_depth=2, n_estimators=27, warm_start=True)

* XGBoost 익스트림 그레이디언트 부스팅

In [97]:
import xgboost
xgb_reg = xgboost .XGBRegressor ()
xgb_reg .fit (X_train, y_train)
y_pred = xgb_reg .predict (X_val)

xgb_reg .fit (X_train, y_train,
eval_set=[ (X_val, y_val )], early_stopping_rounds=2)
y_pred = xgb_reg .predict (X_val)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

[0]	validation_0-rmse:0.43661
[1]	validation_0-rmse:0.39161
[2]	validation_0-rmse:0.38895
[3]	validation_0-rmse:0.38310
[4]	validation_0-rmse:0.38369
[5]	validation_0-rmse:0.37592
[6]	validation_0-rmse:0.37137
[7]	validation_0-rmse:0.36864
[8]	validation_0-rmse:0.37199


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

## 스태킹